In [12]:
import numpy as np
import pandas as pd
import imutils
import pickle
from imutils import paths
from keras.models import load_model
import cv2
from skimage import data, io
import glob
import warnings
import os
import tensorflow as tf
import os.path
warnings.filterwarnings("ignore")

In [13]:
# Enable gpu to grow memory usage. 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
def resize_to_fit(image, width, height):
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))
    return image

In [3]:
number_image_folder = 'test_predict'
model_filename = 'phone_number_model.hdf5'
model_labels_filename = 'model_labels.dat'

In [4]:
with open(model_labels_filename, 'rb') as f:
    lb = pickle.load(f)

In [5]:
model = load_model(model_filename)
number_image_files = glob.glob(os.path.join(number_image_folder, "*"))
print(len(number_image_files))

764


In [6]:
def Check(data):
    if (data.mean() >= 233.75 ):
        return 0
    else:
        return 1

In [7]:
def AddPixels(data, width):
    pixels = []
    for num in range(0, width):
        d = data[:][num]
        is_valid = Check(d)
        if (is_valid == 1):
            pixels.append(num)
    return pixels

In [8]:
def GetChars(pixels):
    chars = []
    number = []
    for num in pixels:
        index = pixels.index(num)
        if(index != len(pixels)-1):
            if (pixels[index + 1] - num == 1 ):
                number.append(num)
            else:
                number.append(num)
                if(len(number) > 10):
                    char1 = []
                    char2 = []
                    for n in range(0, len(number)):
                        if (n< 8):
                            char1.append(number[n])
                        else:
                            char2.append(number[n])
                    chars.append(tuple(char1))
                    chars.append(tuple(char2))
                else:
                    chars.append(tuple(number))
                number.clear()
        else:
            chars.append(tuple(number))
    return chars

In [11]:
correct = 0
incorrect = 0
temp = 'temp/temp.png'
for image_file in number_image_files:
    filename = os.path.basename(image_file)
    photo_correct_text = os.path.splitext(filename)[0]
    phone_mod = ''
    for text in photo_correct_text:
        if (text != ' '):
            phone_mod += text
    image = data.imread(image_file)
    image_pd = pd.DataFrame(image)
    width = image_pd.shape[1]
    pixels = AddPixels(image_pd, width)
    empty = image_pd[width - 1]
    predictions = []
    chars = GetChars(pixels)
    for char in chars:
        ch = list(char)
        char_pd = image_pd[ch]
        for c in ch:
            char_pd[c] = char_pd[c].apply(lambda x : 0 if x < 255 else 255)
        char_pd[len(ch)] = empty
        io.imsave(temp, char_pd)
        num = cv2.imread(temp)
        num = cv2.cvtColor(num, cv2.COLOR_BGR2GRAY)
        num = resize_to_fit(num, 12, 12)
        num = np.expand_dims(num, axis = 2)
        num = np.expand_dims(num, axis = 0)
        pred = model.predict(num)
        letter = lb.inverse_transform(pred)[0]
        predictions.append(letter)
    pred_phone = ''
    for prediction in predictions:
        pred_phone = pred_phone+''+prediction
    if pred_phone in phone_mod:
        print(image_file+':'+pred_phone+',Y')
        correct += 1
    else :
        print(image_file+':'+pred_phone+'N')
        incorrect +=1
print('Correct: {}'.format(correct))
print('Incorrect: {}'.format(incorrect))

test_predict\0221002862.png:0221002862,Y
test_predict\0221755588.png:0221755588,Y
test_predict\0221819999#3269.png:0221819999#3269,Y
test_predict\0221819999.png:0221819999,Y
test_predict\0222037236.png:0222037236,Y
test_predict\0222081888.png:0222081888,Y
test_predict\0222182018.png:0222182018,Y
test_predict\0222235656.png:0222235656,Y
test_predict\0222288844.png:0222288844,Y
test_predict\0222311111.png:0222311111,Y
test_predict\0222482766.png:0222482766,Y
test_predict\0222532087.png:0222532087,Y
test_predict\0222544453.png:0222544453,Y
test_predict\0222547771.png:0222547771,Y
test_predict\0222557733.png:0222557733,Y
test_predict\0222559386.png:0222559386,Y
test_predict\0222570775.png:0222570775,Y
test_predict\0222576445.png:0222576445,Y
test_predict\0222586906.png:0222586906,Y
test_predict\0222632010.png:0222632010,Y
test_predict\0222692125.png:0222692125,Y
test_predict\0222726988.png:0222726988,Y
test_predict\0222765388.png:0222765388,Y
test_predict\0222972666.png:0222972666,Y
test_p

test_predict\0226360459.png:0226360459,Y
test_predict\0226361548.png:0226361548,Y
test_predict\0226361585.png:0226361585,Y
test_predict\0226462266.png:0226462266,Y
test_predict\0226512651.png:0226512651,Y
test_predict\0226562188.png:0226562188,Y
test_predict\0226562752.png:0226562752,Y
test_predict\0226571895.png:0226571895,Y
test_predict\0226573315.png:0226573315,Y
test_predict\0226574837.png:0226574837,Y
test_predict\0226597752.png:0226597752,Y
test_predict\0226709345.png:0226709345,Y
test_predict\0226732161.png:0226732161,Y
test_predict\0226800104.png:0226800104,Y
test_predict\0226844785.png:0226844785,Y
test_predict\0226860356.png:0226860356,Y
test_predict\0226934131.png:0226934131,Y
test_predict\0227001680.png:0227001680,Y
test_predict\0227003501.png:0227003501,Y
test_predict\0227009855.png:0227009855,Y
test_predict\0227012030.png:0227012030,Y
test_predict\0227016669.png:0227016669,Y
test_predict\0227019227.png:0227019227,Y
test_predict\0227019992.png:0227019992,Y
test_predict\022

test_predict\0228823286.png:0228823286,Y
test_predict\0228829393.png:0228829393,Y
test_predict\0228830366.png:0228830366,Y
test_predict\0228831123.png:0228831123,Y
test_predict\0228831599.png:0228831599,Y
test_predict\0228833820.png:0228833820,Y
test_predict\0228835983.png:0228835983,Y
test_predict\0228883322.png:0228883322,Y
test_predict\0228922207.png:0228922207,Y
test_predict\0228937958.png:0228937958,Y
test_predict\0229021169.png:0229021169,Y
test_predict\0229109475.png:0229109475,Y
test_predict\0229277718.png:0229277718,Y
test_predict\0229281300.png:0229281300,Y
test_predict\0229306132.png:0229306132,Y
test_predict\022952339.png:022952339,Y
test_predict\0229525595.png:0229525595,Y
test_predict\0229562124.png:0229562124,Y
test_predict\0229603503.png:0229603503,Y
test_predict\0229612323.png:0229612323,Y
test_predict\0229691587.png:0229691587,Y
test_predict\0229915250.png:0229915250,Y
test_predict\0229965986.png:0229965986,Y
test_predict\0231515666.png:0231515666,Y
test_predict\02332

test_predict\0423014567.png:0423014567,Y
test_predict\0423019859.png:0423019859,Y
test_predict\0423022122.png:0423022122,Y
test_predict\0423027087.png:0423027087,Y
test_predict\0423029599.png:0423029599,Y
test_predict\0423051698.png:0423051698,Y
test_predict\0423056688.png:0423056688,Y
test_predict\0423103189.png:0423103189,Y
test_predict\0423124798.png:0423124798,Y
test_predict\0423158355.png:0423158355,Y
test_predict\0423175895.png:0423175895,Y
test_predict\0423191158.png:0423191158,Y
test_predict\0423191768.png:0423191768,Y
test_predict\0423195101.png:0423195101,Y
test_predict\0423198852.png:0423198852,Y
test_predict\0423201586.png:0423201586,Y
test_predict\0423202090.png:0423202090,Y
test_predict\0423205157.png:0423205157,Y
test_predict\0423208297.png:0423208297,Y
test_predict\0423232979.png:0423232979,Y
test_predict\0423263322.png:0423263322,Y
test_predict\0423266469.png:0423266469,Y
test_predict\0423266984.png:0423266984,Y
test_predict\0423267339.png:0423267339,Y
test_predict\042